In [1]:
# -*- coding: utf-8 -*-
"""
Created on Wed Aug 23 11:14:00 2023

@author: al005366
"""
import logging

# %%
# Import necessary libraries
import os
from datetime import datetime

import matplotlib
import matplotlib.pyplot as plt
import mlflow
import mlflow.sklearn
import numpy as np
import pandas as pd
import statsmodels.api as sm
import xgboost as xgb
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
from sklearn.kernel_ridge import KernelRidge
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import GridSearchCV, TimeSeriesSplit
from sklearn.neural_network import MLPRegressor
from statsmodels.tsa.api import VAR

matplotlib.style.use('ggplot')

# Define the size of the DataFrame
size = 400

In [2]:
# Create a DataFrame with fake data
df = pd.DataFrame({
    'cpi': np.random.rand(size),
    'm1': np.random.rand(size),
    'm3sa': np.random.rand(size),
    'ip': np.random.rand(size),
    'retailSales': np.random.rand(size),
    'gvtYld10y': np.random.rand(size),
    'ShortGovYield': np.random.rand(size),
    'pmiCom': np.random.rand(size),
    'policyRate': np.random.rand(size),
    'UnempRate': np.random.rand(size)
})

# Define the start date and periods based on the size of the DataFrame
start_date = pd.to_datetime('2024-04-01') - pd.DateOffset(months=size-1)

# Create a date column with monthly frequency
df['date'] = pd.date_range(start=start_date, periods=size, freq='M')
df.set_index(df['date'], drop=True, inplace=True)

# Calculate percentage changes and add lag features
cols2calc = ['cpi','m1','m3sa','ip','retailSales']
df[[f'{c}_gr' for c in cols2calc]] = df[cols2calc].pct_change(12, fill_method=None) * 100
df['termSpread'] = df['gvtYld10y'] - df['ShortGovYield']

# Add lag features
df[[c + '_L1' for c in ['cpi_gr']][0]] = df['cpi_gr'].shift(1).copy()
df[[c + '_L2' for c in ['cpi_gr']][0]] = df['cpi_gr'].shift(2).copy()
df[[c + '_F1' for c in ['cpi_gr']][0]] = df['cpi_gr'].shift(-1).copy()
df[[c + '_F3' for c in ['cpi_gr']][0]] = df['cpi_gr'].shift(-3).copy()
df[[c + '_F12' for c in ['cpi_gr']][0]] = df['cpi_gr'].shift(-12).copy()

# Define columns to lag
cols2lag = ['pmiCom', 'policyRate', 'm3sa', 'm1', 'UnempRate', 
            'termSpread','m1_gr', 'm3sa_gr', 'ip_gr', 'retailSales_gr']

In [3]:
# Define model specifications
mod_1 = ['cpi_gr'] + [f'cpi_gr_{l}' for l in ['L1', 'L2']]
mod_2 = mod_1 + ['pmiCom', 'policyRate', 'UnempRate', 'termSpread', 'm1_gr', 'm3sa_gr', 'ip_gr', 'retailSales_gr']
mod_3 = mod_1 + ['policyRate']
mod_4 = mod_1 + ['termSpread']
mod_5 = mod_1 + ['m1_gr']
mod_6 = mod_1 + ['m3sa_gr']
mod_7 = mod_1 + ['UnempRate', 'ip_gr', 'retailSales_gr']

# Store models in a dictionary
models = {
    'model_1': mod_1,
    'model_2': mod_2,
    'model_3': mod_3,
    'model_4': mod_4,
    'model_5': mod_5,
    'model_6': mod_6,
    'model_7': mod_7
}

In [4]:
# Define training and test dates
train_start_date = '2010-03-01'
train_end_date = '2016-12-01'
test_start_date_1 = '2017-01-01'
test_end_date_1 = '2019-12-01'
test_start_date_2 = '2022-01-01'
test_end_date_2 = '2022-09-01'

# Split the data into training and testing sets
df_train = df.loc[(df.index >= train_start_date) & (df.index <= train_end_date), :].copy()
train_x = {mname: pd.DataFrame() for mname in models.keys()}
# Populate train_x with training data
for mname, varsname in models.items():
    train_x[mname] = df_train[varsname].dropna().copy()
    train_x[mname] = sm.add_constant(train_x[mname])

df_test_1 = df.loc[(df.index >= test_start_date_1) & (df.index <= test_end_date_1), :].copy()
test_x_1 = {mname: pd.DataFrame() for mname in models.keys()}
# Populate test_x_1 with test data
for mname, varsname in models.items():
    test_x_1[mname] = df_test_1[varsname].dropna().copy()
    test_x_1[mname] = sm.add_constant(test_x_1[mname])

df_test_2 = df.loc[(df.index >= test_start_date_2) & (df.index <= test_end_date_2), :].copy()
test_x_2 = {mname: pd.DataFrame() for mname in models.keys()}
# Populate test_x_2 with test data
for mname, varsname in models.items():
    test_x_2[mname] = df_test_2[varsname].dropna().copy()
    test_x_2[mname] = sm.add_constant(test_x_2[mname])

df_test = df.loc[(df.index > train_end_date) & (df.index <= test_end_date_2), :].copy()
test_x = {mname: pd.DataFrame() for mname in models.keys()}
# Populate test_x with test data
for mname, varsname in models.items():
    test_x[mname] = df_test[varsname].dropna().copy()
    test_x[mname] = sm.add_constant(test_x[mname])

# Define the target variable
y = df.filter(regex='_F').copy()
train_y = y[y.index.isin(df_train[varsname].dropna().index.tolist())].copy()
test_y_1 = df['cpi_gr'][df.index.isin(df_test_1.index.tolist())].copy()
test_y_2 = df['cpi_gr'][df.index.isin(df_test_2.index.tolist())].copy()
test_y = df['cpi_gr'][df.index.isin(df_test.index.tolist())].copy()

In [5]:
# TimeSeriesSplit for cross-validation
tscv = TimeSeriesSplit(n_splits=5)

# Define models and hyperparameters for benchmarking
ridge_model = Ridge()
ridge_params_grid = {"alpha": [100, 1e0, 1e-5]}

kernel_ridge_model = KernelRidge()
kr_params_grid = {"alpha": [100, 1e0, 1e-5],
                "kernel": ['linear', 'rbf'],
                "gamma": [1e0, 1e-5]}

rf_model = RandomForestRegressor()
rf_params_grid = {
    'bootstrap': [True],
    'max_depth': [80, 110],
    'n_estimators': [100, 1000],
    'random_state': [0]
}

gbr_model = GradientBoostingRegressor()
gbr_params_grid = {
    "n_estimators": [10, 500],
    "learning_rate": [0.0001, 1.0],
    "loss": ["squared_error"],
    "random_state": [0]
}

xgb_model = xgb.XGBRegressor()
xgb_params_grid = {
    'subsample': [1],
    'max_depth': [3, 5],
    'gamma': np.linspace(0, 1, 3),
    'colsample_bytree': [0.15]
}

mlp_model = MLPRegressor(max_iter=1000)
mlp_params_grid = {
    'hidden_layer_sizes': [(50, 50), (100, 200)],
    'activation': ['tanh'],
    'solver': ['lbfgs', 'adam'],
    'alpha': [0.0, 0.05],
    'learning_rate': ['constant'],
}

benchmarking_dict = {
    'rf': {'model': rf_model, 'param_grid': rf_params_grid},
    'ridge':{'model': ridge_model, 'param_grid': ridge_params_grid},
    #'kernel_ridge': {'model': kernel_ridge_model, 'param_grid': kr_params_grid},
    #'gbr':{'model': gbr_model, 'param_grid': gbr_params_grid},
    #'xgb': {'model': xgb_model, 'param_grid': xgb_params_grid},
    'mlp': {'model': mlp_model, 'param_grid': mlp_params_grid}
}

# Specify horizons and metrics
horizon = ['1m', '3m', '12m']
metric = ['r2', 'rmse_1', 'rmse_2']

# Create a dictionary to store results
res = dict()

In [6]:
# Get the current date
today = datetime.today().date().isoformat()
today_str = today.replace('-', '')

# Set the tracking URI to your local tracking server
mlflow.set_tracking_uri("http://localhost:6000")

# Define experiment name
experiment_name = f"{today}-benchmarking-eusdfa-123"

# Check if the experiment exists
experiment = mlflow.get_experiment_by_name(experiment_name)

if experiment:
    experiment_id = experiment.experiment_id
else:
    # Create the experiment
    experiment_id = mlflow.create_experiment(experiment_name)

print(f"Using experiment ID: {experiment_id}")

Using experiment ID: 2


In [ ]:
# Loop through each model for benchmarking
for model in benchmarking_dict.keys():
    # Initialize results dataframe         
    res[model] = pd.DataFrame(index=models.keys(), columns=pd.MultiIndex.from_product([horizon, metric], names=['horizon', 'metric']))
    # Loop through each target horizon
    for h in train_y.columns:
        hh = h.split('_')[-1].split('F')[-1] + 'm'
        # Loop through each model specification
        for modname, modvars in models.items():
            # Start an MLflow run
            with mlflow.start_run(run_name=f"inflation-{today_str}-{model}-{h}-{modname}", experiment_id=experiment_id):
                # Logging information    
                print(f'\n Model: {model} \n Spec: {modname} \n Spec vars: {modvars} \n Horizon: {hh}')
                mlflow.log_param("Model", model)
                mlflow.log_param("Spec", modname)
                mlflow.log_param("Spec_vars", modvars)
                mlflow.log_param("Horizon", hh)
                
                # Grid search for hyperparameter tuning
                model_paramSearch = GridSearchCV(benchmarking_dict[model]['model'], 
                                                param_grid=benchmarking_dict[model]['param_grid'],
                                                cv=tscv)
                model_paramSearch.fit(train_x[modname], train_y[h])
                best_params = model_paramSearch.best_params_
                
                # Fit the model with the best hyperparameters
                res_fit = benchmarking_dict[model]['model'].set_params(**model_paramSearch.best_params_).fit(train_x[modname], train_y[h])

                # Log the resulting model
                mlflow.sklearn.log_model(res_fit, "model")

                # Log the best parameters
                for param, value in best_params.items():
                    mlflow.log_param(param, value)
                
                # Make predictions on test data
                pred_o = res_fit.predict(test_x[modname])
                pred_o_1 = res_fit.predict(test_x_1[modname])
                pred_o_2 = res_fit.predict(test_x_2[modname])
                pred_i = res_fit.predict(train_x[modname])
                
                try:
                    coeff_ = pd.DataFrame(res_fit.coef_, index=train_x[modname].columns, columns=['coeff'])
                except Exception:
                    coeff_ = []
                
                # Calculate evaluation metrics
                r2 = r2_score(train_y[h], pred_i)
                rmse_1 = mean_squared_error(test_y_1, pred_o_1, squared=False)
                rmse_2 = mean_squared_error(test_y_2, pred_o_2, squared=False)
                res[model].loc[modname, (hh, 'r2')] = r2
                res[model].loc[modname, (hh, 'rmse_1')] = rmse_1
                res[model].loc[modname, (hh, 'rmse_2')] = rmse_2

                # Log metrics
                mlflow.log_metric("r2", r2)
                mlflow.log_metric("rmse_1", rmse_1)
                mlflow.log_metric("rmse_2", rmse_2)

                # Save results to a pickle file
                res2pkl = {
                    'fit': res_fit,
                    'pred_o': pred_o,
                    'pred_o_1': pred_o_1,
                    'pred_o_2': pred_o_1,
                    'train_x': train_x[modname],
                    'test_x': test_x[modname],
                    'test_x_1': test_x_1[modname],
                    'test_x_2': test_x_2[modname],
                    'train_y': train_y[h],
                    'test_y': test_y,
                    'test_y_1': test_y_1,
                    'test_y_2': test_y_2,
                    'pred_i': pred_i,
                    'coeff_': coeff_,
                    'r2': res[model].loc[modname, (hh, 'r2')],
                    'rmse_1': res[model].loc[modname, (hh, 'rmse_1')],
                    'rmse_2': res[model].loc[modname, (hh, 'rmse_2')]
                }
            
            #with open(f'{models_folder}{model}\\{model}_{modname}_{hh}.pkl','wb') as f:
            #    pickle.dump(res2pkl, f)
    
    # Save results to an Excel file        
    #res[model].to_excel(res_folder + f'{model}_V2.xlsx', sheet_name='data')

# Save overall results to a pickle file               
#with open(res_folder + '\\results_models_V2.pkl', 'wb') as f:
#    pickle.dump(res, f)

print("Experiment completed and logged.")


 Model: rf 
 Spec: model_1 
 Spec vars: ['cpi_gr', 'cpi_gr_L1', 'cpi_gr_L2'] 
 Horizon: 1m


/opt/conda/lib/python3.11/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")



 Model: rf 
 Spec: model_2 
 Spec vars: ['cpi_gr', 'cpi_gr_L1', 'cpi_gr_L2', 'pmiCom', 'policyRate', 'UnempRate', 'termSpread', 'm1_gr', 'm3sa_gr', 'ip_gr', 'retailSales_gr'] 
 Horizon: 1m

 Model: rf 
 Spec: model_3 
 Spec vars: ['cpi_gr', 'cpi_gr_L1', 'cpi_gr_L2', 'policyRate'] 
 Horizon: 1m

 Model: rf 
 Spec: model_4 
 Spec vars: ['cpi_gr', 'cpi_gr_L1', 'cpi_gr_L2', 'termSpread'] 
 Horizon: 1m

 Model: rf 
 Spec: model_5 
 Spec vars: ['cpi_gr', 'cpi_gr_L1', 'cpi_gr_L2', 'm1_gr'] 
 Horizon: 1m

 Model: rf 
 Spec: model_6 
 Spec vars: ['cpi_gr', 'cpi_gr_L1', 'cpi_gr_L2', 'm3sa_gr'] 
 Horizon: 1m

 Model: rf 
 Spec: model_7 
 Spec vars: ['cpi_gr', 'cpi_gr_L1', 'cpi_gr_L2', 'UnempRate', 'ip_gr', 'retailSales_gr'] 
 Horizon: 1m

 Model: rf 
 Spec: model_1 
 Spec vars: ['cpi_gr', 'cpi_gr_L1', 'cpi_gr_L2'] 
 Horizon: 3m

 Model: rf 
 Spec: model_2 
 Spec vars: ['cpi_gr', 'cpi_gr_L1', 'cpi_gr_L2', 'pmiCom', 'policyRate', 'UnempRate', 'termSpread', 'm1_gr', 'm3sa_gr', 'ip_gr', 'retailSales

/opt/conda/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:546: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
/opt/conda/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:546: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
/opt/conda/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:546: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS R


 Model: mlp 
 Spec: model_2 
 Spec vars: ['cpi_gr', 'cpi_gr_L1', 'cpi_gr_L2', 'pmiCom', 'policyRate', 'UnempRate', 'termSpread', 'm1_gr', 'm3sa_gr', 'ip_gr', 'retailSales_gr'] 
 Horizon: 1m


/opt/conda/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:546: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
/opt/conda/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:546: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
/opt/conda/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:546: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.


 Model: mlp 
 Spec: model_3 
 Spec vars: ['cpi_gr', 'cpi_gr_L1', 'cpi_gr_L2', 'policyRate'] 
 Horizon: 1m


/opt/conda/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:546: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
/opt/conda/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:546: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
/opt/conda/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:546: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS R

In [8]:
# Set the tracking URI, otherwise mlflow will not find your runs
experiment_id = mlflow.get_experiment_by_name(experiment_name).experiment_id
print(experiment_id)
runs = mlflow.search_runs(experiment_ids=[experiment_id])#filter_string=f"tags.mlflow.experiment_name = '{experiment_name}'")
#print(runs)
best_run = runs.loc[runs["metrics.r2"].idxmax()]
print(best_run)
run_id = best_run["run_id"]
# %% Load model for the given run
path = f"runs:/{run_id}/model"
model = mlflow.sklearn.load_model(path)
print(model)

494547387029483515
run_id                                            26e112c387d94d0f9952a66f2118e6c6
experiment_id                                                   494547387029483515
status                                                                    FINISHED
artifact_uri                     mlflow-artifacts:/494547387029483515/26e112c38...
start_time                                        2024-06-08 21:37:33.667000+00:00
end_time                                          2024-06-08 21:37:54.068000+00:00
metrics.rmse_1                                                         1233.385577
metrics.rmse_2                                                         2354.642139
metrics.r2                                                                0.964884
params.Spec_vars                              ['cpi_gr', 'cpi_gr_L1', 'cpi_gr_L2']
params.Model                                                                    rf
params.Horizon                                                      

RandomForestRegressor(max_depth=80, n_estimators=1000, random_state=0)
